# Llama2Chat

This notebook shows how to augment Llama-2 `LLM`s with the `Llama2Chat` wrapper to support the [Llama-2 chat prompt format](https://huggingface.co/blog/llama2#how-to-prompt-llama-2). Several `LLM` implementations in LangChain can be used as interface to Llama-2 chat models. These include [ChatHuggingFace](/docs/integrations/chat/huggingface), [LlamaCpp](/docs/use_cases/question_answering/local_retrieval_qa), [GPT4All](/docs/integrations/llms/gpt4all), ..., to mention a few examples. 

`Llama2Chat` is a generic wrapper that implements `BaseChatModel` and can therefore be used in applications as [chat model](/docs/modules/model_io/chat/). `Llama2Chat` converts a list of Messages into the [required chat prompt format](https://huggingface.co/blog/llama2#how-to-prompt-llama-2) and forwards the formatted prompt as `str` to the wrapped `LLM`.

In [2]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat

For the chat application examples below, we'll use the following chat `prompt_template`:

In [3]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

template_messages = [
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
]
prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [ ]:
# !pip3 install text-generation

Then you are ready to use the chat `model` together with `prompt_template` and conversation `memory` in an `LLMChain`.

In [5]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [6]:
print(
    chain.run(
        text="What can I see in Vienna? Propose a few locations. Names only, no details."
    )
)

 Sure, I'd be happy to help! Here are a few popular locations to consider visiting in Vienna:

1. Schönbrunn Palace
2. St. Stephen's Cathedral
3. Hofburg Palace
4. Belvedere Palace
5. Prater Park
6. Vienna State Opera
7. Albertina Museum
8. Museum of Natural History
9. Kunsthistorisches Museum
10. Ringstrasse


In [7]:
print(chain.run(text="Tell me more about #2."))

 Certainly! St. Stephen's Cathedral (Stephansdom) is one of the most recognizable landmarks in Vienna and a must-see attraction for visitors. This stunning Gothic cathedral is located in the heart of the city and is known for its intricate stone carvings, colorful stained glass windows, and impressive dome.

The cathedral was built in the 12th century and has been the site of many important events throughout history, including the coronation of Holy Roman emperors and the funeral of Mozart. Today, it is still an active place of worship and offers guided tours, concerts, and special events. Visitors can climb up the south tower for panoramic views of the city or attend a service to experience the beautiful music and chanting.


## Chat with Llama-2 via `LlamaCPP` LLM

For using a Llama-2 chat model with a [LlamaCPP](/docs/integrations/llms/llamacpp) `LMM`, install the `llama-cpp-python` library using [these installation instructions](/docs/integrations/llms/llamacpp#installation). The following example uses a quantized [llama-2-7b-chat.Q4_0.gguf](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_0.gguf) model stored locally at `~/Models/llama-2-7b-chat.Q4_0.gguf`. You can download it from  [here](https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/blob/main/llama-2-7b-chat.Q4_0.gguf)

After creating a `LlamaCpp` instance, the `llm` is again wrapped into `Llama2Chat`

In [6]:
from os.path import expanduser

from langchain_community.llms import LlamaCpp

model_path = expanduser("/home/ashok/Models/llama-2-7b-chat.Q4_0.gguf")

llm = LlamaCpp(
                model_path=model_path,
                streaming=False,
                )

model = Llama2Chat(llm=llm)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/ashok/Models/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u3

and used in the same way as in the previous example.

In [7]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [8]:
print(
    chain.invoke(
        text="What can I see in Vienna? Propose a few locations. Names only, no details."
    )
)

/home/ashok/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =     694.35 ms
llama_print_timings:      sample time =      65.24 ms /   135 runs   (    0.48 ms per token,  2069.25 tokens per second)
llama_print_timings: prompt eval time =    3809.89 ms /    47 tokens (   81.06 ms per token,    12.34 tokens per second)
llama_print_timings:        eval time =   22012.89 ms /   134 runs   (  164.28 ms per token,     6.09 tokens per second)
llama_print_timings:       total time =   26254.55 ms /   181 tokens


  Certainly! Vienna is a city with a rich history and culture, offering plenty of exciting places to visit. Here are some popular attractions to consider:
1. Schönbrunn Palace
2. St. Stephen's Cathedral
3. Hofburg Palace
4. Belvedere Palace
5. Prater Park
6. MuseumsQuartier
7. Vienna State Opera
8. Albertina Museum
9. St. Charles Church
10. Naschmarkt

I hope this helps! Let me know if you have any specific interests or preferences, and I can recommend some places based on that.


In [9]:
print(chain.invoke(text="Tell me more about #2."))

Llama.generate: prefix-match hit

llama_print_timings:        load time =     694.35 ms
llama_print_timings:      sample time =     115.65 ms /   230 runs   (    0.50 ms per token,  1988.72 tokens per second)
llama_print_timings: prompt eval time =   12668.41 ms /   151 tokens (   83.90 ms per token,    11.92 tokens per second)
llama_print_timings:        eval time =   38526.56 ms /   229 runs   (  168.24 ms per token,     5.94 tokens per second)
llama_print_timings:       total time =   51930.91 ms /   380 tokens


  Sure! St. Stephen's Cathedral is a beautiful Gothic-style Catholic church located in the heart of Vienna. It's one of the most iconic landmarks in the city and is known for its stunning architecture, including its tall spires and intricate stone carvings.
The cathedral was founded in the 12th century and has been renovated and expanded over the centuries. Today, it's a popular tourist destination and place of worship for both locals and visitors.
Inside the cathedral, you can admire the stunning architecture and see various works of art, including mosaics, frescoes, and sculptures. You can also climb to the top of the South Tower for panoramic views of Vienna from above.
St. Stephen's Cathedral is located in the historic city center of Vienna, near many other popular attractions such as the Hofburg Palace, the State Opera House, and the Ringstrasse boulevard. It's a must-see stop for anyone interested in history, architecture, or religion.
